In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from scipy.sparse import hstack

from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import ast
import re
from urllib.parse import urlparse

In [2]:
# Reliability
def get_username(instagram_url):
    parsed_url = urlparse(instagram_url)
    path_segments = parsed_url.path.strip('/').split('/')
    username = path_segments[-1]
    return username
insta_df = pd.read_csv('../../data/reliability.csv').drop(columns = 'Unnamed: 0').drop_duplicates(subset='username', keep='last')
social_media = pd.read_csv('../../data/credibility_with_insta_username.csv').drop(columns = 'Unnamed: 0')
cleaned_social_media = social_media[social_media['instagram_uri'].str.contains('instagram', na=False)].reset_index().drop(columns = 'index')
cleaned_social_media['username'] = cleaned_social_media['instagram_uri'].apply(get_username)

In [3]:
# Reliability
unique_df = cleaned_social_media[['username', 'source']].drop_duplicates(subset='source', keep='last')
merged_df = pd.merge(insta_df, unique_df, on='username', how='inner').reset_index().drop(columns = 'index')
merged_df.head()

,username,biography,userid,mediacount,followers,followees,is_verified,source
0,scottwalker,Christian American Husband Father 45th Governo...,586553277,8683,47989,175,True,Scott Walker
1,newyorkcitizensauditnews,NY Citizens Audit is a Volunteer based group. ...,50346417000,1202,891,460,False,New York Citizens Audit
2,steventkirsch,Retired serial entrepreneur. Follow me on stev...,184544694,10,4469,20,False,Steve Kirsch
3,shannonrwatts,"🎤 Speaker, author, organizer\n🙋‍♀️ “Summoner o...",24779854,431,129536,2369,True,Shannon Watts
4,chris_larson,NaN,5941863,1138,1743,349,False,Chris Larson


In [ ]:
standard = ['True', 'Mostly True ','Half True ', 'Mostly False', 'False', 'Pants on Fire']
score = np.array([10, 8, 6, 4, 2, 0])
poli_df = pd.read_csv('../../data/politifact_data.csv')
def transform_percentages(col):
    return sum(np.array(ast.literal_eval(col.replace('%', '').replace('\'', '').replace(' ', ',')))/100 * score)
poli_df['credibility_score'] = poli_df['percentages'].apply(transform_percentages)
poli_df['content'] = poli_df['content'].str.replace("“", "").str.replace("”","").str.replace("\"", '')
poli_df = poli_df.rename(columns = {'speaker':'recorder', 'media':'source'})

def extract_date(text):
    date_pattern = re.compile(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b')
    match = date_pattern.search(text) 
    if match:
        date = match.group()
        return date
poli_df['publish_time'] = poli_df['when/where'].apply(extract_date)
poli_df['content'].head()[0]
clean_df = poli_df[['source', 'credibility_score']].drop_duplicates().reset_index().drop(columns = 'index')
clean_df.head()
import requests
def search_wikipedia(query, num_results=15):
    endpoint_url = "https://en.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'srsearch': query,
        'srlimit': num_results
    }
    response = requests.get(endpoint_url, params=params)
    data = response.json()
    output = []
    search_results = data['query']['search']
    for result in search_results:
        title = result['title']
        output.append(title)
    return " ".join(output)
def scrape_titles(col):
    return search_wikipedia(col, num_results=20)
clean_df['source_history'] = clean_df['source'].apply(scrape_titles)
# clean_df.to_csv('../../data/credibility.csv')

In [4]:
clean_df = pd.read_csv('../../data/credibility.csv', index_col=None).drop(columns = 'Unnamed: 0')

In [5]:
clean_df

,source,credibility_score,source_history
0,Instagram posts,1.74,Instagram List of most-followed Instagram acco...
1,Scott Walker,5.14,Scott Walker Scott Walker (politician) Scott W...
2,Viral image,1.60,Viral phenomenon Imgur The dress Midjourney Da...
3,Facebook posts,1.86,Facebook Facebook 3D Posts Meta Platforms Hist...
4,New York Citizens Audit,2.00,Sovereign citizen movement KPMG BNY Mellon Arc...
...,...,...,...
4695,Freedom Defense Advocates,0.00,United States Army Judge Advocate General's Co...
4696,Spike the Romney Attack Dog,8.00,Mitt Romney List of individual dogs January 6 ...
4697,Log Cabin Republicans,8.00,Log Cabin Republicans Don Korotsky Norte Log C...
4698,Sam Brownback,6.50,Sam Brownback Kansas experiment Sam Brownback ...


In [6]:
import sys
cred_reliability = pd.merge(merged_df, clean_df, on='source', how='inner').reset_index().drop(columns='index')
cred_reliability['followees_to_followers_ratio'] = cred_reliability['followees'] / cred_reliability['followers'] + 1e-10
cred_reliability = cred_reliability[['source', 'source_history', 'mediacount', 'followees_to_followers_ratio', 'is_verified', 'credibility_score']]
cred_reliability = cred_reliability.dropna(subset=['source_history'])
cred_reliability = cred_reliability.replace([np.inf, -np.inf], np.nan)
cred_reliability = cred_reliability.dropna(subset=['followees_to_followers_ratio'])
cred_reliability['is_verified'] = cred_reliability['is_verified'].apply(lambda x: 0 if x else 1)
cred_reliability.head()

,source,source_history,mediacount,followees_to_followers_ratio,is_verified,credibility_score
0,Scott Walker,Scott Walker Scott Walker (politician) Scott W...,8683,0.003647,0,5.14
1,New York Citizens Audit,Sovereign citizen movement KPMG BNY Mellon Arc...,1202,0.516274,1,2.00
2,Steve Kirsch,Steve Kirsch Stan Kirsch Against All Odds (198...,10,0.004475,1,0.80
3,Shannon Watts,Shannon Watts Watts family murders Shannon Wat...,431,0.018288,0,6.00
4,Chris Larson,Chris Larson Chris Larson (disambiguation) Bri...,1138,0.200229,1,4.60


In [7]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    tokens = nltk.word_tokenize(text)    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
cred_reliability['preprocessed_source'] = cred_reliability['source_history'].apply(preprocess_text)

In [ ]:
def text_embedding(data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
    
    def get_bert_embeddings(data):
        tokens = tokenizer(data.tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            embeddings = bert_model(**tokens).last_hidden_state.mean(dim=1)
        return embeddings

    batch_size = 128
    num_samples = len(data)
    num_batches = (num_samples + batch_size - 1) // batch_size

    embeddings_list = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = data.iloc[start_idx:end_idx]
        batch_embeddings = get_bert_embeddings(batch_data)
        embeddings_list.append(batch_embeddings)

    embeddings = torch.cat(embeddings_list, dim=0).cpu().numpy()
    return embeddings

In [ ]:
X = cred_reliability['preprocessed_source']
X = text_embedding(X)
X = np.concatenate((X[:, :50], cred_reliability[['mediacount', 'followees_to_followers_ratio', 'is_verified']].values), axis=1)
y = cred_reliability['credibility_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# vectorizer = TfidfVectorizer()
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# X_additional_features_train = cred_reliability.loc[X_train.index, ['followees_to_followers_ratio', 'mediacount', 'is_verified']].values
# X_additional_features_test = cred_reliability.loc[X_test.index, ['followees_to_followers_ratio', 'mediacount', 'is_verified']].values
# X_train_tfidf_with_features = hstack((X_train_tfidf, X_additional_features_train))
# X_test_tfidf_with_features = hstack((X_test_tfidf, X_additional_features_test))


In [ ]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    RandomForestRegressor(),
    BayesianRidge(),
]
for model in models:
    if model.__class__.__name__ != 'BayesianRidge':
        model.fit(X_train_tfidf_with_features, y_train)
        y_pred = model.predict(X_test_tfidf_with_features)
    else:    
        model.fit(X_train_tfidf_with_features.toarray(), y_train)
        y_pred = model.predict(X_test_tfidf_with_features.toarray())
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error for {model.__class__.__name__}: {mse}')